# LM Evaluation Harness

検証は rinna 3.6B を g5.2xlarge にて行った。

In [ ]:
!pip install sagemaker transformers

## Model の用意 (Optional)

学習済みのモデルをローカルにダウンロードする。

モデルは事前学習済みモデル、ファインチューニング/LoRA（Continuous Pretraining / Instruction Tuning）などを使用できます。

[PMC-Llama](https://arxiv.org/abs/2304.14454) などでは対象のタスクでの Fine-tuning も行っています。必要に応じてタスクでの学習も行い性能を比較することも考えられます。

In [ ]:
# アーティファクトのダウンロード
import boto3
import sagemaker

def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

# Job 名 でアーティファクトの S3 URL を取得
model_data = get_latest_training_job_artifact('Rinna')

!aws s3 cp {model_data} model.tar.gz

In [ ]:
# ファイルの解凍

# LoRA の場合は True、Full Model の場合は False
lora = False
model_dir = "downloaded_model"

# フォルダのクリーンアップ
!rm -rf {model_dir} && mkdir {model_dir}

# LoRA の場合は adapter_* など
if lora:
    !tar -xvf model.tar.gz -C {model_dir} --no-same-owner --wildcards adapter_*

# Full Model の場合は pytorch_model* など
else:
    !tar -xvf model.tar.gz -C {model_dir} --no-same-owner --exclude checkpoint*

    # SageMaker Model Parallel などのコードで実行した際の Artifact は fullmodel.pt なので HuggingFace 形式に変換
    !mv {model_dir}/fullmodel.pt {model_dir}/pytorch_model.bin
    import torch
    user_content = torch.load(f"{model_dir}/user_content_fullmodel.pt")
    hf_config = user_content['model_config']
    hf_config.to_json_file(f"{model_dir}/config.json")

    # トークナイザが保存されていない場合はトークナイザを保存する
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        "rinna/japanese-gpt-neox-3.6b-instruction-ppo",
    )
    tokenizer.save_pretrained(model_dir)

## カスタムデータセットの用意 (Optional)

要約のサンプルとして `lm_eval/tasks/ja/xlsum_ja.py` を `lm_eval/tasks/ja/custom_summary.py` にコピーしていくつか書き換えたものがあります。
また、`lm-evaluation-harness/lm_eval/tasks/__init__.py` もそれに合わせて編集しました。

xlsum は Rogue2 を使用した評価を行います。

このデータセットは `text`, `summary` のフィールドのデータセットを受け取り要約します。（例：https://huggingface.co/datasets/mkshing/xlsum_ja ）

`lm_eval/tasks/ja/custom_summary.py` の `DATASET_PATH` を必要に応じて書き換えてください。（ローカルのパスもしくは Huggingface のデータセット ID）

例として Wikihow データセットの変換を行います。

In [ ]:
!mkdir -p downloaded_dataset
import pandas as pd

df = pd.read_csv("~/sagemaker_nlp_examples/Summarization/T5_training/test.csv") # Wikihow データセットが保存されているパス
df.rename(columns={'src': 'text', 'tgt': 'summary'}, inplace=True)
df = df[:100]
df.to_csv("downloaded_dataset/test.csv", index=False)

## 評価の実行

In [ ]:
!pip install -e ".[ja]"
!pip install "accelerate>=0.20.3"

### Full Model

In [ ]:
custom_model_directory = "models/rinna/rinna-japanese-gpt-neox-3.6b-customft"
custom_model_harness = f"{custom_model_directory}/harness.sh"

In [ ]:
!mkdir -p {custom_model_directory}

In [ ]:
# %%writefile {custom_model_harness}
# MODEL_ARGS="pretrained=downloaded_model,use_fast=False,device_map=auto,torch_dtype=auto,batch_size=4"
# TASK="jcommonsenseqa-1.1-0.4,jnli-1.1-0.4,marc_ja-1.1-0.4,jsquad-1.1-0.4,jaqket_v2-0.2-0.4,xlsum_ja-1.0-0.4,xwinograd_ja,mgsm-1.0-0.4,custom-1.0-0.0"
# python main.py --model hf-causal --model_args $MODEL_ARGS --tasks $TASK --num_fewshot "3,3,3,2,1,1,0,5,0" --device "cuda" --output_path "models/rinna/rinna-japanese-gpt-neox-3.6b-customft/result.json"

In [ ]:
%%writefile {custom_model_harness}
MODEL_ARGS="pretrained=downloaded_model,use_fast=False,device_map=auto,torch_dtype=auto,batch_size=4"
TASK="custom-1.0-0.0"
python main.py --model hf-causal --model_args $MODEL_ARGS --tasks $TASK --num_fewshot "0" --device "cuda" --output_path "models/rinna/rinna-japanese-gpt-neox-3.6b-customft/result.json"

In [ ]:
!chmod u+x {custom_model_harness}
!{custom_model_harness}

### LoRA

Full Model との差異として PEFT を読み込む実装がある `hf-causal-experimental` を使用している。

LoRA をマージして Full Model として実行することも可能。

In [ ]:
custom_model_directory = "models/rinna/rinna-japanese-gpt-neox-3.6b-lora"
custom_model_harness = f"{custom_model_directory}/harness.sh"

In [ ]:
!mkdir -p {custom_model_directory}

In [ ]:
%%writefile {custom_model_harness}
MODEL_ARGS="pretrained=rinna/japanese-gpt-neox-3.6b,peft=downloaded_model,use_fast=False,device_map_option=auto,dtype=auto,batch_size=4"
TASK="custom-1.0-0.0"
python main.py --model hf-causal-experimental --model_args $MODEL_ARGS --tasks $TASK --num_fewshot "0" --device "cuda" --output_path "models/rinna/rinna-japanese-gpt-neox-3.6b-lora/result.json"

In [ ]:
!chmod u+x {custom_model_harness}
!{custom_model_harness}